Python API Tutorial - An Introduction to Using APIs

https://www.dataquest.io/blog/python-api-tutorial/

In [26]:
import requests
import requests.auth
import json
import getpass

### Get current position of ISS

In [27]:
# Make a request to get the current (within 5 seconds) position of the ISS from the open notify API
response = requests.get("http://api.open-notify.org/iss-now.json")
response.status_code

200

In [28]:
response.content

b'{"iss_position": {"latitude": "34.6997", "longitude": "-18.9199"}, "message": "success", "timestamp": 1553737188}'

In [29]:
response.json()

{'iss_position': {'latitude': '34.6997', 'longitude': '-18.9199'},
 'message': 'success',
 'timestamp': 1553737188}

### Get details of next pass

In [30]:
# Params to pass to the API.
# This is the Lat/Long of Melbourne, Australia
parameters = {"lat": 37.81, "lon": 144.96}

# Make a get request with parameters
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)
response.status_code

200

In [33]:
response.content

b'{\n  "message": "success", \n  "request": {\n    "altitude": 100, \n    "datetime": 1553737199, \n    "latitude": 37.81, \n    "longitude": 144.96, \n    "passes": 5\n  }, \n  "response": [\n    {\n      "duration": 503, \n      "risetime": 1553752166\n    }, \n    {\n      "duration": 645, \n      "risetime": 1553757862\n    }, \n    {\n      "duration": 562, \n      "risetime": 1553763727\n    }, \n    {\n      "duration": 476, \n      "risetime": 1553769638\n    }, \n    {\n      "duration": 552, \n      "risetime": 1553775470\n    }\n  ]\n}\n'

In [34]:
response.headers

{'Server': 'nginx/1.10.3', 'Date': 'Thu, 28 Mar 2019 01:39:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '520', 'Connection': 'keep-alive', 'Via': '1.1 vegur'}

In [35]:
data = response.json()
data

{'message': 'success',
 'request': {'altitude': 100,
  'datetime': 1553737199,
  'latitude': 37.81,
  'longitude': 144.96,
  'passes': 5},
 'response': [{'duration': 503, 'risetime': 1553752166},
  {'duration': 645, 'risetime': 1553757862},
  {'duration': 562, 'risetime': 1553763727},
  {'duration': 476, 'risetime': 1553769638},
  {'duration': 552, 'risetime': 1553775470}]}

### Get number of people in space

In [36]:
response = requests.get("http://api.open-notify.org/astros.json")
response.status_code

200

In [37]:
data = response.json()
data["number"]

6

There will be more to come as this topic forms appears in a later part of the DataQuest Data Scientist course.

### Reddit, what could go wrong

In my defence at the end of the DQ article they do tell the reader to have a look at reddit, so...

In [38]:
# My initial attempts would return a 429 code, thanks to u/,,xx_th for the quick fix
response = requests.get("http://reddit.com/api/trending_subreddits.json", headers = {'User-agent': 'your bot 0.1'})
response.status_code

200

In [40]:
# From here out any long response strings will be placed in a variable but not printe
# Having huge blocks of json within the notebook as caused issues for me in the past
reddit_headers = response.headers
# reddit_headers

21

In [43]:
reddit_response = response.json()
# reddit_response

Further exploration is limited by my lack of knowledge of oauth and modhashes, this is a problem for another day

### More Reddit starting with OAuth2

I immediatly discovered that Reddit won't let you do much without authentication.

#### Request a token

In [44]:
# Client API data. Apps client ID and apps client secret
# Because i can't be trusted the secret will need to be input each time.
client_secret = getpass.getpass("input client secret from https://www.reddit.com/prefs/apps")
client_auth = requests.auth.HTTPBasicAuth('LJqjGWbNCzUpKg', client_secret)

input client secret from https://www.reddit.com/prefs/apps········


In [47]:
headers = {"User-Agent": "ClientMcClientface by /u/timbo72"}
password = getpass.getpass("Password")
post_data = {"grant_type": "password", "username": "timbo72", "password": password}

Password········


In [61]:
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
token = response.json()['access_token']

In [62]:
headers = {"Authorization": "bearer " + token, "User-Agent": "ClientMcClientface by /u/timbo72"}
response = requests.get("https://oauth.reddit.com/api/v1/me", headers=headers)
my_data = response.json()

In [64]:
headers = {"Authorization": "bearer " + token, "User-Agent": "ClientMcClientface by /u/timbo72"}
response = requests.get("https://oauth.reddit.com/r/PrequelMemes/random", headers=headers)
prequel_memes = response.json()

### Now for something completely different

I've been looking at the 'have i been pwned' API.
First off there is a great script written by Dr Mike Pound, lets give it a go.


In [69]:
import hashlib
import sys

try:
    import requests
except ModuleNotFoundError:
    print("### requests library not found ###")
    raise

    
def lookup_pwned_api(pwd):
    """
    Returns hash and number of times password was seen in pwned database.
    
    Args: pwd: password to check
    
    Returns: A (sha1, count) tuple of the SHA-1 hashed password and the number of times it was found in the database.
    
    Raises: RuntimeError: if there was an error trying to fetch the data from the API.
            UnicodeError: if there was and error UTF_encoding the password.
    """
    sha1pwd = hashlib.sha1(pwd.encode('utf-8')).hexdigest().upper()
    head, tail = sha1pwd[:5], sha1pwd[5:]
    url = "https://api.pwnedpasswords.com/range/" + head
    res = requests.get(url)
    if res.status_code != 200:
        raise RuntimeError("Error fetching '{}': {}".format(url, res.status_code))
    hashes = (line.split(":") for line in res.text.splitlines())
    count = next((int(count) for t, count in hashes if t == tail), 0)
    return sha1pwd, count


def main(args):
    ec = 0
    for pwd in args or sys.stdin:
        pwd = pwd.strip()
        try:
            sha1pwd, count = lookup_pwned_api(pwd)
            
            if count:
                foundmsg = "{0} was found with {1} occurances (hash: {2})"
                print(foundmsg.format(pwd, count, sha1pwd))
                ec = 1
            else:
                print("{} was not found".format(pwd))
        except UnicodeError:
            errormsg = sys.exc_info()[1]
            print("{0} could not be checked: {1}". format(pwd, errormsg))
            ec = 1
            continue
    return ec

import getpass
lookup_pwned_api(getpass.getpass("Password"))

Password········


('7CB9DED83CD7F2704D6435C743ED3124ED56C701', 766)

Lets have a play with V2

In [68]:
import requests

#pwd = input("Input and email to check:")
pwd = "timbo72@gmail.com"
headers = {"api-version":"2", "user-agent":"Timbos-api-notebook"}    
url = "https://haveibeenpwned.com/api/breachedaccount/" + pwd + "?truncateResponse=true"
response = requests.get(url, headers = headers)
#data = response.json()
response.status_code

403

In [23]:
sites = []
for i in data:
    sites.append(i["Name"])
    
sites

['Adobe', 'Disqus', 'Dropbox', 'MyFitnessPal']

### What3Words

Heard about an interesting idea for geolocation called what3words, they have an API...

In [3]:
import requests
import getpass

key = getpass.getpass("API Key")
lat = "-38.619797"
long = "146.124734"
url = "https://api.what3words.com/v3/convert-to-3wa?coordinates=" + lat + "%2c" + long + "&key=" + key
response = requests.get(url)

API Key········


In [4]:
if response.status_code != 200:
    code = response.json()['error']['code']
    message = response.json()['error']['message']
    print(code)
    print(message)
else:
    words = response.json()['words']
    print(words)

called.reef.odds


ToDo:  

Getting sick of having to enter API keys and such manually...  
option a: txt file with whatever i need, read in at start of notebook.
    problen is i can't keep that file in github...  
option b: option a plus encryption...

needs:  
reddit client secret  
reddit client auth  
reddit password  
W3W API key

doesn't need encrytion:   
W3W lat/long

In [23]:
from cryptography.fernet import Fernet
import pickle
import getpass

W3W_lat, W3W_long = '-38.619797', '146.124734'
stuff_in = {'W3W_lat':W3W_lat,'W3W_long':W3W_long}

In [25]:
key = Fernet.generate_key()
cipher_suite = Fernet(key)
cipher_text = cipher_suite.encrypt(W3W_lat.encode())
cipher_text

········


TypeError: generate_key() takes 1 positional argument but 2 were given

In [21]:
plain_text = cipher_suite.decrypt(cipher_text)
stuff_out = plain_text.decode()
stuff_out

'-38.619797'

In [32]:
#def write_to_file(key,value,file):
with open('workfile.pickle','wb') as file:
    pickle.dump(stuff_in, file)

In [35]:
#def read_from_file(file):
with open('workfile.pickle','rb') as file:
    stuff_out = pickle.load(file)

In [37]:
stuff_out['W3W_lat']

'-38.619797'

In [34]:
with open('workfile.txt') as file:
    text_from_file = file.read()

In [25]:
stuff['W3W_API_KEY'] = 'balloon'
stuff

{'W3W_lat': '-38.619797', 'W3W_long': '146.124734', 'W3W_API_KEY': 'balloon'}

In [ ]:
import json
json.dumps(stuff)

In [18]:
with open('workfile.txt', 'a') as file:
    json.dump(stuff,file)

In [19]:
with open('workfile.txt') as file:
    print(file.read())

{"W3W_lat": "-38.619797", "W3W_long": "146.124734"}{"W3W_lat": "-38.619797", "W3W_long": "146.124734"}{"W3W_lat": "-38.619797", "W3W_long": "146.124734"}
